In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("OPENWEATHER_API_KEY")
ACCESS_KEY = os.getenv("LAKEFS_ACCESS_KEY")
SECRET_KEY = os.getenv("LAKEFS_SECRET_KEY")
lakefs_endpoint = os.getenv("LAKEFS_ENDPOINT", "http://lakefs-dev:8000")


In [2]:
# #test read parquet
# import pandas as pd
# storage_options = {
#     "key": ACCESS_KEY,
#     "secret": SECRET_KEY,
#     "client_kwargs": {
#         "endpoint_url": lakefs_endpoint
#     }
# }

# repo = "pollution-data"
# branch = "main"
# path = "pollution.parquet"
# lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

In [5]:
# path_partition = 's3a://pollution-data/main/pollution.parquet/'
# # path_partition = 's3a://pollution-data/main/pollution.parquet/year=2025/month=5/day=11/hour=10/5127aa0fd46841dcba3ad95008af1c7d-0.parquet'
# df = pd.read_parquet(
#     path=path_partition,
#     storage_options=storage_options,
#     engine="pyarrow"
#     )
# df.info()
# df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180522 entries, 0 to 180521
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype                       
---  ------            --------------   -----                       
 0   timestamp         180522 non-null  datetime64[ns]              
 1   minute            180522 non-null  int64                       
 2   localtime         180522 non-null  datetime64[ns, Asia/Bangkok]
 3   province          180522 non-null  string                      
 4   district          180522 non-null  string                      
 5   district_id       180522 non-null  int64                       
 6   lat               180522 non-null  float64                     
 7   lon               180522 non-null  float64                     
 8   main.aqi          180522 non-null  int64                       
 9   components_co     180522 non-null  float64                     
 10  components_no     180522 non-null  float64              

,timestamp,minute,localtime,province,district,district_id,lat,lon,main.aqi,components_co,...,components_o3,components_so2,components_pm2_5,components_pm10,components_nh3,flow_timestamp,year,month,day,hour
0,2025-05-13 10:00:09.082896,0,2025-05-13 17:00:09.082896+07:00,Krabi,Nuea Khlong,8108,8.0747,99.0036,1,109.01,...,39.42,0.02,1.79,2.08,0.00,2025-05-13 10:00:08.202481,2025,5,13,10
1,2025-05-13 10:00:08.950969,0,2025-05-13 17:00:08.950969+07:00,Krabi,Lam Thap,8107,8.0717,99.2917,1,104.79,...,38.10,0.02,1.73,2.02,0.01,2025-05-13 10:00:08.202481,2025,5,13,10
2,2025-05-13 10:00:08.949743,0,2025-05-13 17:00:08.949743+07:00,Krabi,Plai Phraya,8106,8.5333,98.8628,1,125.19,...,41.21,0.10,2.72,3.11,0.04,2025-05-13 10:00:08.202481,2025,5,13,10
3,2025-05-13 10:00:08.872550,0,2025-05-13 17:00:08.872550+07:00,Krabi,Khao Phanom,8102,8.2647,99.0492,1,115.44,...,42.29,0.02,2.19,2.35,0.00,2025-05-13 10:00:08.202481,2025,5,13,10
4,2025-05-13 10:00:09.088553,0,2025-05-13 17:00:09.088553+07:00,Krabi,Ao Luek,8105,8.3775,98.7217,1,125.97,...,43.89,0.05,3.24,3.51,0.00,2025-05-13 10:00:08.202481,2025,5,13,10


In [4]:
import s3fs
path = 's3a://pollution-data/main/pollution.parquet'
fs = s3fs.S3FileSystem(
    key=ACCESS_KEY,
    secret=SECRET_KEY,
    client_kwargs={'endpoint_url': lakefs_endpoint}
)

data_list = fs.glob(f"{path}/*/*/*/*/*")
len(data_list)

540

In [5]:
import pyarrow.dataset as ds
path = "pollution-data/main/pollution.parquet/"
#path = "weather-data/main/weather.parquet/"
dataset = ds.dataset(
    path,
    format="parquet",
    partitioning="hive",
    filesystem=fs
)

table = dataset.to_table()
df = table.to_pandas()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490140 entries, 0 to 490139
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype                       
---  ------            --------------   -----                       
 0   timestamp         490140 non-null  datetime64[ns]              
 1   minute            490140 non-null  int64                       
 2   localtime         490140 non-null  datetime64[ns, Asia/Bangkok]
 3   province          490140 non-null  string                      
 4   district          490140 non-null  string                      
 5   district_id       490140 non-null  int64                       
 6   lat               490140 non-null  float64                     
 7   lon               490140 non-null  float64                     
 8   main.aqi          490140 non-null  int64                       
 9   components_co     490140 non-null  float64                     
 10  components_no     490140 non-null  float64              

In [25]:
from datetime import datetime

filtered = dataset.to_table(
    filter=(
        (ds.field("year") == 2025) &
        (ds.field("month") == 5) &
        (
        # (ds.field("day") == 16) |
        # (ds.field("day") == 17) |
        (ds.field("day") == 18)
        ) &
        (ds.field("hour") == 12)
    )
)
# df_filtered = filtered.to_pandas()
# df_filtered

pollution_df_filtered = filtered.to_pandas()
pollution_df_filtered
# weather_df_filtered = filtered.to_pandas()
# weather_df_filtered

,timestamp,minute,localtime,province,district,district_id,lat,lon,main.aqi,components_co,...,components_o3,components_so2,components_pm2_5,components_pm10,components_nh3,flow_timestamp,year,month,day,hour
0,2025-05-18 12:45:01.050238,45,2025-05-18 19:45:01.050238+07:00,Krabi,Nuea Khlong,8108,8.0747,99.0036,1,143.28,...,42.92,0.11,3.64,4.42,0.08,2025-05-18 12:45:00.171361,2025,5,18,12
1,2025-05-18 12:45:01.022953,45,2025-05-18 19:45:01.022953+07:00,Krabi,Lam Thap,8107,8.0717,99.2917,1,156.39,...,43.94,0.13,3.82,4.47,0.19,2025-05-18 12:45:00.171361,2025,5,18,12
2,2025-05-18 12:45:01.049276,45,2025-05-18 19:45:01.049276+07:00,Krabi,Plai Phraya,8106,8.5333,98.8628,1,140.46,...,37.98,0.20,3.06,3.88,0.13,2025-05-18 12:45:00.171361,2025,5,18,12
3,2025-05-18 12:45:00.998138,45,2025-05-18 19:45:00.998138+07:00,Krabi,Khao Phanom,8102,8.2647,99.0492,1,147.06,...,43.47,0.14,3.80,4.58,0.10,2025-05-18 12:45:00.171361,2025,5,18,12
4,2025-05-18 12:45:01.003746,45,2025-05-18 19:45:01.003746+07:00,Krabi,Ao Luek,8105,8.3775,98.7217,1,103.28,...,33.87,0.07,2.13,2.91,0.03,2025-05-18 12:45:00.171361,2025,5,18,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3703,2025-05-18 12:33:18.285919,33,2025-05-18 19:33:18.285919+07:00,Mae Hong Son,Khun Yuam,5802,18.8394,97.9514,1,242.43,...,17.93,0.34,8.13,9.62,1.35,2025-05-18 12:29:59.790098,2025,5,18,12
3704,2025-05-18 12:33:18.291932,33,2025-05-18 19:33:18.291932+07:00,Mae Hong Son,Pai,5803,19.3586,98.4400,1,186.33,...,23.11,0.65,3.39,3.63,0.56,2025-05-18 12:29:59.790098,2025,5,18,12
3705,2025-05-18 12:33:18.283726,33,2025-05-18 19:33:18.283726+07:00,Mae Hong Son,Mae Sariang,5804,18.1594,97.9336,1,254.05,...,30.52,0.42,6.60,8.28,1.78,2025-05-18 12:29:59.790098,2025,5,18,12
3706,2025-05-18 12:33:18.459686,33,2025-05-18 19:33:18.459686+07:00,Mae Hong Son,Mae La Noi,5805,18.3844,97.9369,1,308.23,...,26.91,0.54,9.88,12.13,2.09,2025-05-18 12:29:59.790098,2025,5,18,12


In [14]:
# merge
import pandas as pd
weather_pollution_df = pd.merge(
    pollution_df_filtered,
    weather_df_filtered,
    on=['flow_timestamp', 'district_id', 'lat', 'lon'],
    suffixes=('_pollution', '_weather')
)

weather_pollution_df['province'] = weather_pollution_df['province_pollution']
weather_pollution_df['district'] = weather_pollution_df['district_pollution']

weather_pollution_df['year'] = weather_pollution_df['year_pollution']
weather_pollution_df['month'] = weather_pollution_df['month_pollution']
weather_pollution_df['day'] = weather_pollution_df['day_pollution']
weather_pollution_df['hour'] = weather_pollution_df['hour_pollution']


### main pollution
weather_pollution_df['timestamp'] = weather_pollution_df['timestamp_pollution']
weather_pollution_df['localtime'] = weather_pollution_df['localtime_pollution']
weather_pollution_df['minute'] = weather_pollution_df['minute_pollution']

weather_pollution_df.drop(columns=[
    'province_pollution', 'province_weather',
    'district_pollution', 'district_weather',
    'timestamp_pollution', 'timestamp_weather',
    'minute_pollution', 'minute_weather',
    'localtime_pollution', 'localtime_weather',
    'year_pollution', 'year_weather',
    'month_pollution', 'month_weather',
    'day_pollution', 'day_weather',
    'hour_pollution', 'hour_weather'
], inplace=True)


weather_pollution_df

,district_id,lat,lon,main.aqi,components_co,components_no,components_no2,components_o3,components_so2,components_pm2_5,...,wind.deg,province,district,year,month,day,hour,timestamp,localtime,minute
0,8108,8.0747,99.0036,2,257.44,0.00,0.94,79.18,0.14,12.92,...,178,Krabi,Nuea Khlong,2025,5,16,10,2025-05-16 10:45:01.067003,2025-05-16 17:45:01.067003+07:00,45
1,8107,8.0717,99.2917,2,266.39,0.00,1.16,79.48,0.22,13.37,...,178,Krabi,Lam Thap,2025,5,16,10,2025-05-16 10:45:01.049242,2025-05-16 17:45:01.049242+07:00,45
2,8106,8.5333,98.8628,2,284.02,0.00,1.64,76.96,0.22,11.30,...,301,Krabi,Plai Phraya,2025,5,16,10,2025-05-16 10:45:00.885766,2025-05-16 17:45:00.885766+07:00,45
3,8102,8.2647,99.0492,2,268.83,0.00,0.84,83.16,0.12,13.03,...,182,Krabi,Khao Phanom,2025,5,16,10,2025-05-16 10:45:01.055475,2025-05-16 17:45:01.055475+07:00,45
4,8105,8.3775,98.7217,2,234.54,0.00,0.68,80.67,0.07,12.03,...,195,Krabi,Ao Luek,2025,5,16,10,2025-05-16 10:45:01.056543,2025-05-16 17:45:01.056543+07:00,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256226,5802,18.8394,97.9514,1,195.77,0.02,1.04,35.18,0.21,5.96,...,246,Mae Hong Son,Khun Yuam,2025,5,18,9,2025-05-18 09:33:24.909839,2025-05-18 16:33:24.909839+07:00,33
256227,5803,19.3586,98.4400,1,160.92,0.10,0.74,30.68,0.27,2.73,...,277,Mae Hong Son,Pai,2025,5,18,9,2025-05-18 09:33:24.908107,2025-05-18 16:33:24.908107+07:00,33
256228,5804,18.1594,97.9336,1,221.16,0.08,1.36,43.21,0.35,7.47,...,268,Mae Hong Son,Mae Sariang,2025,5,18,9,2025-05-18 09:33:24.913289,2025-05-18 16:33:24.913289+07:00,33
256229,5805,18.3844,97.9369,1,241.23,0.07,1.80,42.05,0.39,8.88,...,277,Mae Hong Son,Mae La Noi,2025,5,18,9,2025-05-18 09:33:24.911613,2025-05-18 16:33:24.911613+07:00,33


In [13]:
print(pollution_df_filtered.columns.tolist())

# weather_pollution_df

['timestamp', 'minute', 'localtime', 'province', 'district', 'district_id', 'lat', 'lon', 'main.aqi', 'components_co', 'components_no', 'components_no2', 'components_o3', 'components_so2', 'components_pm2_5', 'components_pm10', 'components_nh3', 'flow_timestamp', 'year', 'month', 'day', 'hour']


In [22]:
# pollution_df_filtered = pollution_df_filtered.drop(columns=['main.aqi'])
# pollution_df_filtered = 
# cols_to_optimize = [
#     'components_co', 'components_no', 'components_no2', 'components_o3',
#     'components_so2', 'components_pm2_5', 'components_pm10', 'components_nh3',
#     'lat', 'lon'
# ]

# ปัดทศนิยม + แปลงเป็น float32
# pollution_df_filtered[cols_to_optimize] = pollution_df_filtered[cols_to_optimize].round(2).astype("float32")

pollution_df_filtered.info()

# df_filtered["day"].unique()
# df_filtered["day"].value_counts()
# weather_pollution_df["day"].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260487 entries, 0 to 260486
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype                       
---  ------            --------------   -----                       
 0   timestamp         260487 non-null  datetime64[ns]              
 1   minute            260487 non-null  int64                       
 2   localtime         260487 non-null  datetime64[ns, Asia/Bangkok]
 3   province          260487 non-null  string                      
 4   district          260487 non-null  string                      
 5   district_id       260487 non-null  int64                       
 6   lat               260487 non-null  float64                     
 7   lon               260487 non-null  float64                     
 8   main.aqi          260487 non-null  int64                       
 9   components_co     260487 non-null  float64                     
 10  components_no     260487 non-null  float64              

In [26]:
#save
# pollution_df_filtered.to_parquet("pollution161718.parquet", index=False)
pollution_df_filtered.to_csv('pollution161718.csv', index=False)


In [21]:
import pandas as pd
df = pd.read_parquet("weatherpollution161718.parquet", engine="pyarrow")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256231 entries, 0 to 256230
Data columns (total 33 columns):
 #   Column               Non-Null Count   Dtype                       
---  ------               --------------   -----                       
 0   district_id          256231 non-null  int64                       
 1   lat                  256231 non-null  float64                     
 2   lon                  256231 non-null  float64                     
 3   main.aqi             256231 non-null  int64                       
 4   components_co        256231 non-null  float64                     
 5   components_no        256231 non-null  float64                     
 6   components_no2       256231 non-null  float64                     
 7   components_o3        256231 non-null  float64                     
 8   components_so2       256231 non-null  float64                     
 9   components_pm2_5     256231 non-null  float64                     
 10  components_pm10     